In [6]:
from dataset import get_area, get_border
from pinn import PINNModel
from models import model1
from accuracy import AccuracyCalc
import tensorflow as tf

In [7]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [8]:
grid_size = (50, 50)
plotting_grid_size = (200, 200)
x_limits = (0, 2)
y_limits = (0, 2)
border_grid_size = (50, 50)

EPOCHS = 3000
EPRINT = 500
LOSS = 1e-2  # 1e-4

In [9]:
area = get_area(grid_size, x_limits, y_limits)
border = get_border(border_grid_size, x_limits, y_limits)

model = PINNModel(model1(), optm = tf.keras.optimizers.Adam(learning_rate=0.001))
model.train(1, area, border, EPOCHS, LOSS, EPRINT)

(2304,) (2304,)
epoch: 0 loss: 101.467392
epoch: 500 loss: 0.476724029
epoch: 1000 loss: 0.141053617
epoch: 1500 loss: 0.0595020801
epoch: 2000 loss: 0.0323401354
epoch: 2500 loss: 0.0214601979


In [10]:
import numpy as np


def real_u(area):
    area = np.array(area)
    if len(area.shape) >= 3:
        x = area[0]
        y = area[1]
        return np.sin(np.pi * x) * np.sin(np.pi * y)
    else:
        return np.array([np.sin(np.pi * x) * np.sin(np.pi * y) for x, y in area])

# pred_coord = list()
# for _x in x[0]:
#     for _y in x[0]:
#         pred_coord.append([_x, _y])
# pred_u = model.predict(np.array(pred_coord)).ravel().reshape(plotting_grid_size[0], plotting_grid_size[1])
calc = AccuracyCalc(model._model, real_u, area=area)
print("Error:", calc.mse())
print("Error:", calc.maxe())

I0000 00:00:1728243151.661560   28383 service.cc:146] XLA service 0x72d3d0004e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728243151.661626   28383 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-06 22:32:31.693591: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


I0000 00:00:1728243152.939624   28383 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Error: 0.0003414330504463797
Error: 0.07801216336974383
